In [26]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [28]:
test_path = "data/test.csv"
sample_sub_path = "data/sample_submission.csv"
pretrained_weight = "pretrained_weight.h5"

In [29]:
test = pd.read_csv(test_path)
test = test.iloc[:test.shape[0], 2:test.shape[1]]
# Fill NaN
for i in test.columns:
    null_cols = test[test[i].isna()].index
    for n in null_cols:
        test.loc[n,i]=test[i].mean()

In [30]:
# Map the value in attribute_0 and attribute_1
attribute_0 = {"material_5": "0", "material_7": "1"}
attribute_1 = {"material_5": "0", "material_6": "1",
               "material_7": "2", "material_8": "3"}
# Replace the value in attribute_0 and attribute_1
test = test.replace({"attribute_0": attribute_0})
test = test.replace({"attribute_1": attribute_1})
# Change the column type from obejct to int
test["attribute_0"] = test["attribute_0"].astype(int)
test["attribute_1"] = test["attribute_1"].astype(int)

In [31]:
test.head()

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,119.57,0,1,6,4,6,9,6,19.305,10.178,...,18.654,10.802,15.909,18.070000,13.772,13.659,16.825,13.742,17.710,634.612
1,113.51,0,1,6,4,11,8,0,17.883,11.927,...,19.368,12.032,13.998,18.846571,12.473,17.468,16.708,14.776,14.102,537.037
2,112.16,0,1,6,4,8,12,4,18.475,10.481,...,17.774,11.743,17.046,18.086000,10.907,13.363,15.737,17.065,16.021,658.995
3,112.72,0,1,6,4,8,11,10,16.518,10.888,...,18.948,11.790,18.165,16.163000,10.933,15.501,15.667,12.620,16.111,594.301
4,208.00,0,1,6,4,14,16,8,17.808,12.693,...,19.141,12.370,14.578,17.849000,11.941,16.070,16.183,13.324,17.150,801.044


In [32]:
def build_model(load_weight):
    model = keras.models.Sequential()
    model.add(layers.Flatten(input_shape=(23,)),)
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(250, activation='relu'))
    model.add(layers.Dense(125, activation='relu'))
    model.add(layers.Dense(62, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.load_weights(load_weight)
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [33]:
model = build_model(pretrained_weight)

In [34]:
y_pred = model.predict(test.values.tolist())
y_pred = np.array(y_pred)
y_pred = pd.DataFrame(y_pred, columns=["failure"])
submission = pd.read_csv(sample_sub_path)
sub = pd.DataFrame({'id': submission.id, 'failure': y_pred.failure})
print(sub)
sub.to_csv("109550117.csv", index=False)

650/650 [==============================] - 1s 1ms/step
          id   failure
0      26570  0.203205
1      26571  0.183827
2      26572  0.191707
3      26573  0.199716
4      26574  0.291268
...      ...       ...
20770  47340  0.226648
20771  47341  0.142090
20772  47342  0.131994
20773  47343  0.211086
20774  47344  0.163187

[20775 rows x 2 columns]
